# Arquitectura de Streaming en tiempo real. 
#### Autores: Yahya El Baroudi, Samuel Corrionero, Ismael González y Jairo Farfán. 


### 1. Introducción y Objetivos
El objetivo de esta práctica es implementar una arquitectura de **Big Data en Streaming** capaz de ingerir, procesar y analizar datos generados en tiempo real.

Hemos simulado un entorno de red social (similar a Twitter/X) para detectar **Trending Topics** (hashtags más populares) cada minuto. Para ello, utilizamos las siguientes tecnologías:
* **Docker:** Para la virtualización de la infraestructura.
* **Apache Kafka:** Como broker de mensajería para desacoplar el productor del consumidor.
* **Apache Spark (Structured Streaming):** Para el procesamiento de datos utilizando la API de **Dataframes**, tal y como se especifica en el temario.

### 2. Infraestructura (Docker)
Desplegamos un clúster de Kafka y Zookeeper utilizando `docker-compose`. Esto garantiza la portabilidad del proyecto y permite levantar los servicios necesarios sin instalaciones complejas en el sistema operativo anfitrión.

In [ ]:
import time

print("🏗️ Levantando infraestructura Docker...")
!docker compose -f docker/docker-compose.yml up -d

print("⏳ Esperando 30 segundos a que Kafka arranque...")
time.sleep(30) # Damos tiempo a que "caliente"
print("✅ Infraestructura lista.")

🏗️ Levantando infraestructura Docker...
[+] Running 0/1
 ⠋ Network docker_default  Creating                                        0.1s 
[+] Running 0/1
 ⠋ Network docker_default  Creating                                        0.1s 
[+] Running 2/3
 ✔ Network docker_default  Created                                         0.1s 
 ✔ Container zookeeper     Created                                         0.1s 
 ⠋ Container kafka         Creating                                        0.0s 
[+] Running 2/3
 ✔ Network docker_default  Created                                         0.1s 
 ✔ Container zookeeper     Created                                         0.1s 
 ⠋ Container kafka         Creating                                        0.0s 
[+] Running 2/3
 ✔ Network docker_default  Created                                         0.1s 
 ⠙ Container zookeeper     Starting                                        0.2s 
 ✔ Container kafka         Created                                    

### 3. Ingesta de Datos (El Productor)

Para simular el flujo de datos, utilizamos un script en Python (`producer.py`) que actúa como generador de eventos.

* **Funcionamiento:** Genera tweets sintéticos en formato JSON con campos aleatorios (usuario, texto, hashtag).
* **Ejecución en Notebook:** Dado que el productor se ejecuta en un bucle infinito, utilizamos la librería `subprocess` para lanzarlo en un hilo en segundo plano (background). Esto permite que el Notebook siga disponible para ejecutar las celdas de Spark sin bloquearse.

In [31]:

import subprocess

print("twitter_simulator: 🚀 Iniciando simulador de tweets en segundo plano...")
# Esto lanza el script sin bloquear la celda
productor_process = subprocess.Popen(["python", "src/productor.py"])
print(f"✅ Productor corriendo (PID: {productor_process.pid})")

twitter_simulator: 🚀 Iniciando simulador de tweets en segundo plano...
✅ Productor corriendo (PID: 35624)


🔄 Iniciando simulador de Twitter hacia localhost:9092...
📩 Enviando: qué complicado es configurar #Streaming
📩 Enviando: qué complicado es configurar #Streaming
📩 Enviando: estoy aprendiendo mucho con #IA
📩 Enviando: estoy aprendiendo mucho con #IA
📩 Enviando: repasando conceptos de #Examen
📩 Enviando: repasando conceptos de #Examen


### 4.1. Configuración de Librerías y Dependencias

En este primer paso, importamos las funciones necesarias de **PySpark SQL** para trabajar con Dataframes.

Un punto crítico aquí es la configuración dinámica del entorno: detectamos la versión de Spark instalada y forzamos la descarga del paquete `.jar` (`spark-sql-kafka`) necesario para conectar Spark con Kafka, ya que este conector no viene instalado por defecto. También preparamos el directorio donde guardaremos los resultados (`data/`).

In [ ]:
import os
import sys
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, current_timestamp, window, count, max as max_
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import pyspark

# Configuración del entorno: Descarga automática del conector de Kafka
spark_version = pyspark.__version__.split("+")[0]
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.12:{spark_version} pyspark-shell'

# Creamos directorio para persistencia de datos
output_dir = Path("data")
output_dir.mkdir(exist_ok=True)

📩 Enviando: mañana tengo examen de #IA
📡 Escuchando... La tabla aparecerá abajo 👇
📩 Enviando: increíble la velocidad de #Examen
📩 Enviando: increíble la velocidad de #Examen
📩 Enviando: mañana tengo examen de #Spark
📩 Enviando: mañana tengo examen de #Spark
📩 Enviando: mañana tengo examen de #IA
📩 Enviando: mañana tengo examen de #IA
📩 Enviando: estoy aprendiendo mucho con #Kafka
📩 Enviando: estoy aprendiendo mucho con #Kafka
📩 Enviando: estoy aprendiendo mucho con #IA
📩 Enviando: estoy aprendiendo mucho con #IA
📩 Enviando: increíble la velocidad de #Python
📩 Enviando: increíble la velocidad de #Python
📩 Enviando: estoy aprendiendo mucho con #BigData
📩 Enviando: estoy aprendiendo mucho con #BigData
📩 Enviando: repasando conceptos de #Examen
📩 Enviando: repasando conceptos de #Examen
📩 Enviando: estoy aprendiendo mucho con #Streaming
📩 Enviando: estoy aprendiendo mucho con #Streaming
📩 Enviando: estoy aprendiendo mucho con #RealTime
📩 Enviando: estoy aprendiendo mucho con #RealTime
📩 En

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/cofer/anaconda3/envs/arqesp/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cofer/anaconda3/envs/arqesp/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cofer/anaconda3/envs/arqesp/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt



🛑 Simulador detenido por el usuario.


KeyboardInterrupt: 

### 4.2. Inicialización de la SparkSession

La `SparkSession` es el punto de entrada para programar con la API de Dataframes y Datasets.

Configuramos el `master` como `local[*]` para utilizar todos los núcleos de la CPU disponibles en la máquina. Además, ajustamos `shuffle.partitions` a 2 (por defecto son 200). Esto es una optimización crucial para entornos locales con pocos datos, ya que evita crear cientos de tareas vacías innecesarias.

In [ ]:
# Iniciar la sesión de Spark
spark = SparkSession.builder \
    .appName("JupyterStreaming") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

# Reducimos el nivel de log para mantener la salida limpia
spark.sparkContext.setLogLevel("ERROR")

### 4.3. Conexión y Lectura desde Kafka

Utilizamos `spark.readStream` para establecer una conexión continua con el broker de Kafka. Esto crea un *Streaming DataFrame*, que representa una tabla de datos infinita que crece constantemente.

* **bootstrap.servers:** Apunta a nuestro contenedor Docker (`localhost:9092`).
* **subscribe:** Escucha el tópico `tweets_topic` donde el Productor está escribiendo.
* **startingOffsets:** Configurado en `latest` para leer solo los datos nuevos y no procesar todo el historial desde el principio.

In [ ]:
df_raw = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "tweets_topic") \
    .option("startingOffsets", "latest") \
    .load()

### 4.4. Estructuración de Datos (Schema Enforcement)

Kafka envía los datos en formato binario (bytes). Para aplicar la lógica de **Spark Dataframes**, debemos transformar estos bytes en una estructura tabular con columnas tipadas.

Definimos un `StructType` que coincide con el JSON generado por nuestro productor (`usuario`, `texto`, `hashtag`, `timestamp`). Usamos la función `from_json` para parsear la columna y `select` para aplanar la estructura.

In [ ]:
# Definición estricta del esquema de datos
schema = StructType([
    StructField("usuario", StringType(), True),
    StructField("texto", StringType(), True),
    StructField("hashtag_principal", StringType(), True),
    StructField("timestamp", DoubleType(), True),
])

# Transformación: De binario -> JSON -> Columnas
df_parsed = df_raw.selectExpr("CAST(value AS STRING) as json_str") \
    .select(from_json(col("json_str"), schema).alias("data")) \
    .select("data.*") \
    .where(col("hashtag_principal").isNotNull()) \
    .withColumn("ts", current_timestamp())

### 4.5. Agregación por Ventanas de Tiempo

Aquí reside la lógica principal del Streaming. En lugar de contar hashtags desde el inicio de los tiempos, utilizamos operaciones de ventana (`window`).

Agrupamos los datos en intervalos de **60 segundos** basándonos en la hora de llegada (`ts`). Esto nos permite calcular los *Trending Topics* dinámicamente minuto a minuto. El `watermark` nos ayuda a gestionar datos que llegan con un ligero retraso, descartando aquellos que sean demasiado viejos.

In [ ]:
conteo = df_parsed \
    .withWatermark("ts", "1 minutes") \
    .groupBy(window(col("ts"), "60 seconds"), col("hashtag_principal")) \
    .agg(count("*").alias("total")) \
    .orderBy(col("total").desc())

### 4.6. Función de Salida Personalizada (ForeachBatch)

Spark Streaming procesa los datos en micro-lotes (*micro-batches*). Definimos una función `mostrar_en_jupyter` que se ejecutará cada vez que un micro-lote esté listo.

Esta función realiza dos tareas:
1.  **Persistencia:** Guarda el lote procesado como un archivo CSV en el disco para auditoría.
2.  **Visualización:** Convierte los datos a un DataFrame de Pandas para mostrarlos de forma tabular y estética en el Notebook.

In [ ]:
def mostrar_en_jupyter(batch_df, batch_id):
    if batch_df.isEmpty():
        return
    
    # Conversión a Pandas para visualización
    pdf = batch_df.limit(10).toPandas()
    
    print(f"📊 Actualización del Stream - Batch ID: {batch_id}")
    
    # Guardado en disco
    csv_path = output_dir / f"batch_{batch_id}.csv"
    pdf.to_csv(csv_path, index=False)
    print(f"Guardado en {csv_path}")
    
    # Impresión de tabla
    print(pdf.to_string(index=False))

### 4.7. Ejecución del Query

Finalmente, iniciamos el flujo de procesamiento.
* **outputMode("complete"):** Indicamos que queremos ver la tabla completa recalculada en cada actualización (necesario para agregaciones con ordenamiento).
* **trigger:** Spark intentará procesar nuevos datos cada 5 segundos.
* **awaitTermination:** Mantiene la celda ejecutándose indefinidamente para escuchar nuevos datos.

In [ ]:
print("📡 Escuchando... La tabla aparecerá abajo 👇")

query = conteo.writeStream \
    .outputMode("complete") \
    .foreachBatch(mostrar_en_jupyter) \
    .trigger(processingTime="5 seconds") \
    .start()

# NOTA: Pulsa el botón "Stop" (Cuadrado) en el menú superior para detener la ejecución.
query.awaitTermination()

### 4.8. Apagamos el productor

In [ ]:

# Ejecuta esto SOLO cuando hayas parado la celda anterior
print("🛑 Apagando el productor...")
productor_process.kill()


🛑 Apagando el productor...


### 5. Persistencia y Auditoría

Además de la visualización en tiempo real, el sistema persiste los resultados procesados en disco. Cada lote procesado se guarda como un archivo CSV en la carpeta `data/`.

A continuación, leemos estos archivos generados para verificar que el histórico de tendencias se ha almacenado correctamente.

### 📂 Visualización de batches grabados
Los archivos CSV generados por cada batch se guardan en `data/` con el nombre `batch_<runId>.csv`. La siguiente celda lista esos archivos y los carga con pandas para que puedas inspeccionar los resultados después de parar el stream.

In [34]:
from pathlib import Path
import pandas as pd

print("📂 Batches guardados:")
for path in sorted(Path("data").glob("batch_*.csv")):
    print(f"- {path.name}")
    display(pd.read_csv(path))

📂 Batches guardados:
- batch_1.csv


,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Spark,1


- batch_10.csv


,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Examen,11
1,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#IA,10
2,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Spark,10
3,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#BigData,9
4,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Streaming,8
5,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#RealTime,7
6,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Kafka,7
7,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Python,6
8,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Kafka,3
9,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#IA,3


- batch_11.csv


...

,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Examen,10
1,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#BigData,7
2,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#IA,7
3,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Streaming,6
4,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Kafka,6
5,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#RealTime,5
6,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Spark,5
7,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Python,4
8,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Kafka,3
9,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#IA,3


- batch_8.csv


,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Examen,11
1,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#BigData,8
2,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Streaming,8
3,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#IA,8
4,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Kafka,6
5,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Spark,6
6,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Python,6
7,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#RealTime,5
8,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Kafka,3
9,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#IA,3


- batch_9.csv


,window,hashtag_principal,total
0,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Examen,11
1,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#IA,9
2,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#BigData,8
3,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Streaming,8
4,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#RealTime,7
5,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Kafka,7
6,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Spark,7
7,"Row(start=Timestamp('2025-12-15 18:40:00'), en...",#Python,6
8,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#Kafka,3
9,"Row(start=Timestamp('2025-12-15 18:39:00'), en...",#IA,3


### 5. Desconexión de infraestructura. 

In [35]:

print("🛑 Apagando infraestructura...")
!docker compose -f docker/docker-compose.yml down

🛑 Apagando infraestructura...
[+] Running 0/1
 ⠋ Container kafka  Stopping                                               0.1s 
[+] Running 0/1
 ⠋ Container kafka  Stopping                                               0.1s 
[+] Running 0/1
 ⠙ Container kafka  Stopping                                               0.2s 
[+] Running 0/1
 ⠙ Container kafka  Stopping                                               0.2s 
[+] Running 0/1
 ⠹ Container kafka  Stopping                                               0.3s 
[+] Running 0/1
 ⠹ Container kafka  Stopping                                               0.3s 
[+] Running 0/1
 ⠸ Container kafka  Stopping                                               0.4s 
[+] Running 0/1
 ⠸ Container kafka  Stopping                                               0.4s 
[+] Running 0/1
 ⠼ Container kafka  Stopping                                               0.5s 
[+] Running 0/1
 ⠼ Container kafka  Stopping                                               0.5s 


25/12/15 18:44:03 ERROR MicroBatchExecution: Query [id = d0068d75-9763-4318-8c6a-f65a68286287, runId = 4f546360-88a1-4ca3-87d6-46eaefb32fd7] terminated with error
java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.TimeoutException: Timed out waiting for a node assignment. Call: describeTopics
	at java.base/java.util.concurrent.CompletableFuture.reportGet(CompletableFuture.java:396)
	at java.base/java.util.concurrent.CompletableFuture.get(CompletableFuture.java:2073)
	at org.apache.kafka.common.internals.KafkaFutureImpl.get(KafkaFutureImpl.java:165)
	at org.apache.spark.sql.kafka010.ConsumerStrategy.retrieveAllPartitions(ConsumerStrategy.scala:66)
	at org.apache.spark.sql.kafka010.ConsumerStrategy.retrieveAllPartitions$(ConsumerStrategy.scala:65)
	at org.apache.spark.sql.kafka010.SubscribeStrategy.retrieveAllPartitions(ConsumerStrategy.scala:102)
	at org.apache.spark.sql.kafka010.SubscribeStrategy.assignedTopicPartitions(ConsumerStrategy.scala:113)
	at org.apache.sp